# 💜 Violet Model Merge - Interactive Notebook

This notebook contains examples of all the different model merging techniques available in **Violet Model Merge**, with explanations and ready-to-run code.

*Derived from [Chattiori Model Merger](https://github.com/faildes) by Chattiori*

## 🎯 Quick Start Guide

1. **Configure Your Paths** (Cell 2) - Set up your model and VAE directories
2. **Discover Your Models** (Cell 3) - Auto-detect available models in your directories  
3. **Choose Your Models** (Cell 4) - Select models for merging using friendly names
4. **Run Merge Examples** - Execute any of the merge technique examples below

## 📋 Table of Contents

### 🛠️ Setup & Configuration
- [📂 Step 1: Configure Your Paths](#configure-paths) — Set up model and VAE directories
- [🔍 Step 2: Discover Available Models](#discover-models) — Auto-scan your model collection  
- [🎯 Step 3: Select Your Models](#select-models) — Choose models for merging
- [🛠️ Merge Engine Setup](#merge-engine) — Initialize the merging functions

### 🎨 Model Merging Techniques
- [1️⃣ WS - Weighted Sum](#ws-merge) — Simple linear blending (2 models)
- [2️⃣ SIG - Sigmoid Merge](#sig-merge) — Smooth non-linear blending (2 models)  
- [3️⃣ GEO - Geometric Merge](#geo-merge) — Geometric mean blending (2 models)
- [4️⃣ MAX - Maximum Merge](#max-merge) — Element-wise maximum (2 models)
- [5️⃣ AD - Add Difference](#ad-merge) — Structure-preserving merge (3 models)
- [6️⃣ sAD - Smooth Add Difference](#sad-merge) — Smoothed structure merge (3 models)
- [7️⃣ MD - Multiply Difference](#md-merge) — Multiplicative blending (3 models)
- [8️⃣ TRS - Triple Sum](#trs-merge) — Advanced triple blending (3 models)
- [9️⃣ DARE - DARE Merge](#dare-merge) — Experimental DARE algorithm (2 models)

### 💅 VAE Enhancement
- [💅 VAE Merging](#vae-merging) — Enhance generation quality with VAE blends

---

## 📂 Step 1: Configure Your Paths {#configure-paths}

First, let's set up your environment paths. **Edit these paths to match your setup:**

In [1]:
import subprocess
import os
import datetime
import threading
import queue
from pathlib import Path

# 🔧 USER CONFIGURATION - Edit these paths to match your setup
# ================================================================

# Path to your models directory (where your .safetensors and .ckpt files are)
models_path = "../../ComfyUI/models/checkpoints"  # 👈 EDIT THIS PATH

# Path to your VAE directory  
vae_path = "../../ComfyUI/models/vae"  # 👈 EDIT THIS PATH

# Python executable (auto-detected, but you can override if needed)
python_exe = None  # Leave as None for auto-detection, or set custom path

# ================================================================
# 🤖 AUTO-CONFIGURATION (No need to edit below this line)
# ================================================================

# Auto-detect Python executable if not specified
if python_exe is None:
    # Try to find the venv python first, then fall back to system python
    # Get the current working directory and look for venv
    current_dir = Path.cwd()
    venv_python = current_dir / "venv" / "Scripts" / "python.exe"
    
    if venv_python.exists():
        python_exe = str(venv_python)
    else:
        # Try some common venv locations
        possible_venvs = [
            current_dir / ".venv" / "Scripts" / "python.exe",
            current_dir / "env" / "Scripts" / "python.exe",
            Path.home() / "venv" / "Scripts" / "python.exe"
        ]
        
        for venv_path in possible_venvs:
            if venv_path.exists():
                python_exe = str(venv_path)
                break
        else:
            # Fall back to system python
            python_exe = "python"

print(f"🐍 Using Python: {python_exe}")
print(f"📁 Models directory: {models_path}")
print(f"🎨 VAE directory: {vae_path}")

# Verify paths exist
if not os.path.exists(models_path):
    print(f"⚠️  Models path doesn't exist: {models_path}")
    print("   Please update the models_path variable above.")
else:
    print(f"✅ Models path verified")

if not os.path.exists(vae_path):
    print(f"⚠️  VAE path doesn't exist: {vae_path}")
    print("   Please update the vae_path variable above.")
else:
    print(f"✅ VAE path verified")

🐍 Using Python: c:\Projects\Violet-Model-Merge\venv\Scripts\python.exe
📁 Models directory: ../../ComfyUI/models/checkpoints
🎨 VAE directory: ../../ComfyUI/models/vae
✅ Models path verified
✅ VAE path verified


## 📂 Step 2: Discover Available Models {#discover-models}

Let's scan your directories and see what models and VAEs are available:

In [3]:
# 🔍 Auto-discover available models and VAEs
def scan_models(directory, extensions=['.safetensors', '.ckpt']):
    """Scan directory for model files"""
    models = []
    if os.path.exists(directory):
        for file in os.listdir(directory):
            if any(file.lower().endswith(ext) for ext in extensions):
                models.append(file)
    return sorted(models)

def display_models(models, title, max_display=10):
    """Display models in a nice format"""
    print(f"\n{title}")
    print("=" * len(title))
    if not models:
        print("  No models found")
        return
    
    for i, model in enumerate(models[:max_display], 1):
        print(f"  {i:2d}. {model}")
    
    if len(models) > max_display:
        print(f"  ... and {len(models) - max_display} more")
    
    print(f"  Total: {len(models)} models")

# Scan for available models
available_models = scan_models(models_path)
available_vaes = scan_models(vae_path, ['.safetensors', '.ckpt', '.pt'])

# Display what we found
display_models(available_models, "🎨 Available Checkpoint Models")
display_models(available_vaes, "🖼️  Available VAE Models")

print(f"\n💡 Found {len(available_models)} checkpoint models and {len(available_vaes)} VAE models")
print("   Use the model selection cell below to choose your models for merging.")


🎨 Available Checkpoint Models
   1. DARE_test.safetensors
   2. Illustrious-XL-v2.0.safetensors
   3. Violet-Delights-1-0.safetensors
   4. grandDesignILXL20_v1.safetensors
   5. ilustmix_v55.safetensors
   6. ilustmix_v6.safetensors
   7. ilustmix_v9.safetensors
   8. plantMilkModelSuite_walnut.safetensors
  Total: 8 models

🖼️  Available VAE Models
   1. Ft-Mse-840000-Ema-Pruned.vae.safetensors
   2. anime_blend_vae.safetensors
   3. anything-v4.0.vae.pt
   4. custom_comp_vae.safetensors
   5. kl-F8-Anime2.vae.safetensors
   6. test_vae_merge.safetensors
  Total: 6 models

💡 Found 8 checkpoint models and 6 VAE models
   Use the model selection cell below to choose your models for merging.


## 🎯 Step 3: Select Your Models {#select-models}

Choose your models for merging. You can either:
- **Option A**: Use the index numbers from the list above
- **Option B**: Specify exact filenames (useful if you know what you want)

In [4]:
# 🎯 MODEL SELECTION - Choose your models for merging
# ================================================================

# Method 1: Select by index number (from the list above)
# Example: If "realism_model.safetensors" is #3 in the list, use model_idx_1 = 3
model_idx_1 = 4  # 👈 EDIT: Index of your first model (1-based)
model_idx_2 = 8  # 👈 EDIT: Index of your second model (1-based)  
model_idx_3 = 2  # 👈 EDIT: Index of your third model (optional, for 3-model merges)

# Method 2: Or specify exact filenames (uncomment and edit these lines)
# model_1 = "your_first_model.safetensors"    # 👈 EDIT: Exact filename
# model_2 = "your_second_model.safetensors"   # 👈 EDIT: Exact filename
# model_3 = "your_third_model.safetensors"    # 👈 EDIT: Exact filename (optional)

# VAE Selection (choose by index or filename)
vae_idx = 3  # 👈 EDIT: Index of your VAE model (1-based)
# vae_model = "your_vae.safetensors"  # 👈 Or specify exact VAE filename

# ================================================================
# 🤖 AUTO-ASSIGNMENT (Don't edit below this line)
# ================================================================

def get_model_by_index(models_list, index, model_type="model"):
    """Get model filename by index (1-based)"""
    if index <= 0 or index > len(models_list):
        print(f"⚠️  Invalid {model_type} index {index}. Available: 1-{len(models_list)}")
        return None
    return models_list[index - 1]

# Assign models by index if not manually specified
if 'model_1' not in locals():
    model_1 = get_model_by_index(available_models, model_idx_1, "model_1")
if 'model_2' not in locals():
    model_2 = get_model_by_index(available_models, model_idx_2, "model_2")
if 'model_3' not in locals():
    model_3 = get_model_by_index(available_models, model_idx_3, "model_3")

# Assign VAE by index if not manually specified
if 'vae_model' not in locals():
    vae_model = get_model_by_index(available_vaes, vae_idx, "VAE")

# Display selected models
print("🎯 Selected Models for Merging:")
print("=" * 35)
print(f"  Model 1: {model_1}")
print(f"  Model 2: {model_2}")
print(f"  Model 3: {model_3}")
print(f"  VAE:     {vae_model}")

# Verify selections
if model_1 and model_1 in available_models:
    print(f"✅ Model 1 verified")
else:
    print(f"❌ Model 1 not found: {model_1}")

if model_2 and model_2 in available_models:
    print(f"✅ Model 2 verified")
else:
    print(f"❌ Model 2 not found: {model_2}")

if vae_model and vae_model in available_vaes:
    print(f"✅ VAE verified")
else:
    print(f"❌ VAE not found: {vae_model}")

print(f"\n💡 Ready to merge! Use any of the merge examples below.")

🎯 Selected Models for Merging:
  Model 1: grandDesignILXL20_v1.safetensors
  Model 2: plantMilkModelSuite_walnut.safetensors
  Model 3: Illustrious-XL-v2.0.safetensors
  VAE:     anything-v4.0.vae.pt
✅ Model 1 verified
✅ Model 2 verified
✅ VAE verified

💡 Ready to merge! Use any of the merge examples below.


## 🛠️ Merge Engine Setup {#merge-engine}

Setting up the core merge functions (no configuration needed):

In [5]:
# 🛠️ MERGE ENGINE FUNCTIONS
# ================================================================
# These functions handle the actual merging process
# You don't need to edit anything in this cell

def clear_merge_log():
    """Clear the last_merge.log file at the start of each run"""
    try:
        with open("last_merge.log", "w", encoding="utf-8") as f:
            f.write(f"=== Merge Log Started at {datetime.datetime.now()} ===\n\n")
    except Exception:
        pass

def log_error(error_type, message, is_blocking=False):
    """Log errors to file and display clean message to user"""
    status = "🚫 Blocking" if is_blocking else "⚠️ Non-blocking"
    
    # Write detailed error to log file
    try:
        with open("last_merge.log", "a", encoding="utf-8") as f:
            f.write(f"[{datetime.datetime.now()}] {status}: {error_type}\n")
            f.write(f"Details: {message}\n")
            f.write("-" * 50 + "\n\n")
    except Exception:
        pass
    
    # Show clean message to user
    print(f"{status} {error_type}: {message.split(':')[-1].strip()}")
    print("📝 See last_merge.log for more details.")

def stream_output_safely(process, output_queue):
    """Safely stream subprocess output, handling encoding issues"""
    try:
        # Try UTF-8 first, then fall back to system encoding with error handling
        for line in process.stdout:
            try:
                decoded_line = line.decode('utf-8')
            except UnicodeDecodeError:
                try:
                    decoded_line = line.decode('cp1252', errors='replace')
                except UnicodeDecodeError:
                    decoded_line = line.decode('utf-8', errors='replace')
            output_queue.put(('stdout', decoded_line))
    except Exception as e:
        log_error("UnicodeDecodeError", str(e), is_blocking=False)
    finally:
        output_queue.put(('done', None))

# Result class for merge operations
class Result:
    def __init__(self, success, return_code, output_lines, cmd_str):
        self.success = success or (return_code == 0)
        self.return_code = return_code
        self.output = "\n".join(output_lines)
        self.command = cmd_str
        
    def __repr__(self):
        status = "✅ Success" if self.success else f"❌ Failed (code {self.return_code})"
        return f"MergeResult({status})"

# Helper function to run merge commands
def run_merge(mode, model0, model1, model2=None, alpha=0.5, beta=0.5, output_name="merged", **kwargs):
    """
    Run a merge operation with the specified parameters
    
    Args:
        mode: Merge algorithm (WS, SIG, GEO, etc.)
        model0: First model filename
        model1: Second model filename  
        model2: Third model filename (optional)
        alpha: Blend ratio between model0 and model1 (0.0-1.0)
        beta: Blend ratio for model2 in 3-model merges (0.0-1.0)
        output_name: Name for the output file
        **kwargs: Additional merge parameters (device, cosine flags, etc.)
    """
    # Clear log at start of each merge
    clear_merge_log()
    
    cmd = [
        python_exe, "violet_merge.py", "model",
        mode, models_path, model0, model1
    ]
    
    # Add model_2 if provided
    if model2:
        cmd.extend(["--model_2", model2])
    
    # Add alpha and beta
    cmd.extend(["--alpha", str(alpha)])
    if beta != 0.5 or mode in ["TRS", "ST", "TS", "SIM", "MD", "DARE"]:
        cmd.extend(["--beta", str(beta)])
    
    # Add VAE path if we have a VAE selected
    if vae_model:
        cmd.extend(["--vae", f"{vae_path}/{vae_model}"])
    
    # Add standard flags
    cmd.extend(["--prune", "--save_half", "--save_safetensors"])
    
    # Default to CUDA device for GPU acceleration (can be overridden)
    if "device" not in kwargs:
        kwargs["device"] = "cuda"
    
    # Add output name
    cmd.extend(["--output", output_name])
    
    # Add any additional kwargs
    for key, value in kwargs.items():
        if isinstance(value, bool) and value:
            cmd.append(f"--{key}")
        else:
            cmd.extend([f"--{key}", str(value)])
    
    # Show command being run
    cmd_str = " ".join(cmd)
    print(f"🚀 Starting merge: {cmd_str}")
    
    # Track merge state
    merge_success = False
    output_lines = []
    
    try:
        # Use subprocess for better output handling
        process = subprocess.Popen(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            universal_newlines=False,  # We'll handle encoding ourselves
            bufsize=1,
            cwd=current_dir
        )
        
        # Create output queue for thread communication
        output_queue = queue.Queue()
        
        # Start output streaming in a separate thread
        output_thread = threading.Thread(
            target=stream_output_safely, 
            args=(process, output_queue)
        )
        output_thread.start()
        
        # Process output in real time
        while True:
            try:
                msg_type, line = output_queue.get(timeout=0.1)
                if msg_type == 'done':
                    break
                elif msg_type == 'stdout':
                    # Clean up the line and display it
                    clean_line = line.rstrip()
                    if clean_line:
                        print(clean_line)
                        output_lines.append(clean_line)
                        
                        # Check for success indicators
                        if "Model saved successfully" in clean_line or "Merge complete" in clean_line:
                            merge_success = True
                            
            except queue.Empty:
                # Check if process is still running
                if process.poll() is not None:
                    break
        
        # Wait for thread to finish
        output_thread.join(timeout=2)
        
        # Get final return code
        return_code = process.wait()
        
        result = Result(merge_success, return_code, output_lines, cmd_str)
        
        if result.success:
            print(f"\n✅ Merge completed successfully!")
            print(f"📁 Output saved as: {output_name}.safetensors")
        else:
            print(f"\n❌ Merge failed with return code: {return_code}")
            log_error("MergeFailure", f"Command failed: {cmd_str}", is_blocking=True)
            
        return result
        
    except FileNotFoundError as e:
        error_msg = f"Could not find merge script: {e}"
        log_error("FileNotFoundError", error_msg, is_blocking=True)
        print(f"🚫 {error_msg}")
        return Result(False, -1, [], cmd_str)
        
    except Exception as e:
        error_msg = f"Unexpected error during merge: {e}"
        log_error("UnexpectedError", error_msg, is_blocking=True)
        print(f"🚫 {error_msg}")
        return Result(False, -1, [], cmd_str)

print("✅ Merge engine functions loaded successfully!")

✅ Merge engine functions loaded successfully!


# 🎨 Merge Technique Examples

Now that your models are configured, you can run any of these merge examples! Each section explains a different merging algorithm and provides ready-to-run code.

**Your current selection:**
- **Model 1**: Selected in step 3 above
- **Model 2**: Selected in step 3 above  
- **VAE**: Selected in step 3 above

---

## 1️⃣ **WS - Weighted Sum (2 Models)** {#ws-merge}

In [ ]:
# Example: Weighted Sum merge
result = run_merge(
    mode="WS",
    model0=model_1,
    model1=model_2,
    alpha=0.4,
    output_name="WS_example"
)

    "## 2️⃣ **SIG - Sigmoid Merge (2 Models)** {#sig-merge}\n",

### 🛠️ How It Works

- Uses a **sigmoid curve function** to blend models.
- It **favors details from the stronger model** more than Weighted Sum.

### ✅ When to Use

- If you want to preserve more detail from **one dominant model**, but still have a **touch** of the other model.
- Great for preserving **fine textures or shading from one source**.

### ⚠️ Why Use It

- Provides a more **organic blend**.
- Useful when one model is **higher quality or more detailed**, and you don't want to lose that.

In [ ]:
# Sigmoid merge
result = run_merge(
    mode="SIG",
    model0=model_1,
    model1=model_2,
    alpha=0.4,
    output_name="SIG_example"
)

## 3️⃣ **GEO - Geometric Merge (2 Models)** {#geo-merge}

### 🛠️ How It Works

In [ ]:
# Geometric merge
result = run_merge(
    mode="GEO",
    model0=model_1,
    model1=model_2,
    alpha=0.4,
    output_name="GEO_example"
)

## 4️⃣ **MAX - Maximum Merge (2 Models)** {#max-merge}

### 🛠️ How It Works

- Picks the **maximum value from each model's weights**.
- This strongly favors **the stronger model** in each part of the latent space.

### ✅ When to Use

In [ ]:
# Maximum merge
result = run_merge(
    mode="MAX",
    model0=model_1,
    model1=model_2,
    alpha=0.4,
    output_name="MAX_example"
)

## 5️⃣ **AD - Add Difference (3 Models)** {#ad-merge}

### 🛠️ How It Works

- Uses **Model 3** as a "difference model" to guide the blend between **Model 1** and **Model 2**.
- This lets you **add features from Model 3 without fully blending it**.

In [8]:
# Add Difference merge (3 models)
result = run_merge(
    mode="AD",
    model0=model_1,
    model1=model_2,
    model2=model_3,
    alpha=0.4,
    output_name="AD_example"
)

🚀 Starting merge: c:\Projects\Violet-Model-Merge\venv\Scripts\python.exe violet_merge.py model AD ../../ComfyUI/models/checkpoints plantMilkModelSuite_walnut.safetensors grandDesignILXL20_v1.safetensors --model_2 ilustmix_v55.safetensors --alpha 0.4 --vae ../../ComfyUI/models/vae/anything-v4.0.vae.pt --prune --save_half --save_safetensors --output AD_example --device cuda
💜 Routing to model merger...


C:\Libraries\Python\Lib\subprocess.py:1018: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)


🎨 Starting Add Difference merge...
📥 Loading plantMilkModelSuite_walnut...
📂 Loading model: plantMilkModelSuite_walnut.safetensors
✅ Model loaded successfully: 2515 keys
🚨 Merge failed: too many values to unpack (expected 5)
🚨 Merge failed: too many values to unpack (expected 5)
Traceback (most recent call last):
  File "c:\Projects\Violet-Model-Merge\violet_merge.py", line 100, in <module>
    main()
  File "c:\Projects\Violet-Model-Merge\violet_merge.py", line 85, in main
    model_main()
  File "c:\Projects\Violet-Model-Merge\lib\merge_model.py", line 1067, in main
    merger.merge_models()
  File "c:\Projects\Violet-Model-Merge\lib\merge_model.py", line 628, in merge_models
    models = self._load_models()
             ^^^^^^^^^^^^^^^^^^^
  File "c:\Projects\Violet-Model-Merge\lib\merge_model.py", line 677, in _load_models
    theta_0, sha256_0, hash_0, meta_0, self.cache_data = load_model(
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: too many values to unpack

## 6️⃣ **sAD - Smooth Add Difference (3 Models)** {#sad-merge}

### 🛠️ How It Works

- Similar to **AD**, but with smoother interpolation between models.

In [ ]:
# Smooth Add Difference merge (3 models)
result = run_merge(
    mode="sAD",
    model0=model_1,
    model1=model_2,
    model2=model_3,
    alpha=0.55,
    beta=0.35,
    output_name="sAD_example"
)

## 7️⃣ **MD - Multiply Difference (3 Models + Beta)** {#md-merge}

### 🛠️ How It Works

- Multiplies the differences between **Model 1 and 2**, then adds them to **Model 3**.
- **Beta** controls how strongly Model 3 influences the final mix.
- **Requires all 3 models** - this is a true 3-model merge algorithm.

### ✅ When to Use

- When Model 3 acts as a **refinement base**, and you want to **multiply stylistic differences** into it.
- Example: Apply the stylistic contrast between **anime and realism models** into a **semi-realistic base model**.
- Perfect for **amplifying differences** between two styles while using a third as the foundation.

### 💡 Key Point

MD **requires exactly 3 models** to function - it calculates differences between models 1 & 2, then applies those multiplied differences to model 3 as the base.

In [ ]:
# Multiply Difference merge (3 models with beta)
result = run_merge(
    mode="MD",
    model0=model_1,
    model1=model_2,
    model2=model_3,
    alpha=0.5,
    beta=0.3,
    output_name="MD_example"
)

🚀 Starting merge: c:\Projects\Violet-Model-Merge\venv\Scripts\python.exe violet_merge.py model MD ../../ComfyUI/models/checkpoints grandDesignILXL20_v1.safetensors plantMilkModelSuite_walnut.safetensors --model_2 Illustrious-XL-v2.0.safetensors --alpha 0.5 --beta 0.3 --vae ../../ComfyUI/models/vae/anything-v4.0.vae.pt --prune --save_half --save_safetensors --output MD_example --device cuda
💜 Routing to model merger...
🎨 Starting Multiply Difference merge...
📥 Loading grandDesignILXL20_v1...
📂 Loading model: grandDesignILXL20_v1.safetensors
🎨 Starting Multiply Difference merge...
📥 Loading grandDesignILXL20_v1...
📂 Loading model: grandDesignILXL20_v1.safetensors
✅ Model loaded successfully: 2515 keys
✅ Model loaded successfully: 2515 keys
📥 Loading plantMilkModelSuite_walnut...
📂 Loading model: plantMilkModelSuite_walnut.safetensors
📥 Loading plantMilkModelSuite_walnut...
📂 Loading model: plantMilkModelSuite_walnut.safetensors
✅ Model loaded successfully: 2515 keys
✅ Model loaded succes

## 8️⃣ **TRS - Triple Sum (3 Models + Beta)** {#trs-merge}

### 🛠️ How It Works

- Averages 3 models, with **beta controlling how much Model 3 contributes**.
- Simple 3-way average.

In [ ]:
# Triple Sum merge (3 models with beta)
result = run_merge(
    mode="TRS",
    model0=model_1,
    model1=model_2,
    model2=model_3,
    alpha=0.33,
    beta=0.33,
    output_name="TRS_example"
)

## 9️⃣ **DARE - DARE Merge (2 Models)** {#dare-merge}

### 🛠️ How It Works

- Applies **advanced blending techniques** combining 2 models.
- Designed for **experimental blending**.

### ✅ When to Use

In [14]:
# DARE merge (experimental blending)
result = run_merge(
    mode="DARE",
    model0=model_1,
    model1=model_2,
    alpha=0.4,
    beta=0.3,  # DARE requires beta parameter
    output_name="DARE_example"
)

🚀 Starting merge: c:\Projects\Violet-Model-Merge\venv\Scripts\python.exe violet_merge.py model DARE ../../ComfyUI/models/checkpoints plantMilkModelSuite_walnut.safetensors grandDesignILXL20_v1.safetensors --alpha 0.4 --beta 0.3 --vae ../../ComfyUI/models/vae/anything-v4.0.vae.pt --prune --save_half --save_safetensors --output DARE_example --device cuda
💜 Routing to model merger...
🎨 Starting DARE merge...
📥 Loading plantMilkModelSuite_walnut...
📂 Loading model: plantMilkModelSuite_walnut.safetensors
🎨 Starting DARE merge...
📥 Loading plantMilkModelSuite_walnut...
📂 Loading model: plantMilkModelSuite_walnut.safetensors
✅ Model loaded successfully: 2515 keys
✅ Model loaded successfully: 2515 keys
📥 Loading grandDesignILXL20_v1...
📂 Loading model: grandDesignILXL20_v1.safetensors
📥 Loading grandDesignILXL20_v1...
📂 Loading model: grandDesignILXL20_v1.safetensors
✅ Model loaded successfully: 2515 keys
✅ Model loaded successfully: 2515 keys
📥 Loading VAE anything-v4.0.vae...
📂 Loading model

## 💡 **Summary & Tips**

### 📝 **Quick Reference Guide**

✅ **For stability and quality:** Use **WS**, **GEO**, or **SIG**.  
✅ **For artistic fusion:** Use **AD**, **sAD**, or **TD**.  
✅ **For experimental creativity:** Use **MD**, **DARE**, or **SIM**.

### 🔧 **Key Parameters**

- **Alpha**: Controls the ratio between model 0 and model 1
  - `0.3` = 70% model 0, 30% model 1
  - `0.7` = 30% model 0, 70% model 1
  
- **Beta**: Controls how much model 2 influences the mix (for 3-model merges)
  - `0.35` = 65% (model 0 + model 1), 35% model 2

### ⚙️ **Standard Flags Used**

- `--prune`: Strips models of original VAE and metadata (always recommended)
- `--save_half`: Saves as FP16 format (most common)
- `--save_safetensors`: Saves as .safetensors format (recommended)
- `--vae`: Adds SDXL VAE to the merged model

### 🚩 **Complete Flag Reference**

#### **Boolean Flags** (Use as `flag_name=True`)

- `force=True` - **Force overwrite** existing output files
- `delete_source=True` - **Delete source** checkpoint files after merging (keeps VAE)
- `no_metadata=True` - **Save without metadata** (smaller file size)
- `keep_ema=True` - **Keep EMA weights** in the output
- `save_quarter=True` - **Save as float8** (experimental, smaller file)

#### **Model Structure Flags** (Choose only one)

- `cosine0=True` - **Favor model 0's structure** with details from others
- `cosine1=True` - **Favor model 1's structure** with details from others  
- `cosine2=True` - **Favor model 2's structure** with details from others

#### **Model Difference Flags** (Advanced users)

- `use_dif_10=True` - Use **difference between model 0 and 1** as model 1
- `use_dif_20=True` - Use **difference between model 0 and 2** as model 2
- `use_dif_21=True` - Use **difference between model 2 and 1** as model 2

#### **Value Flags** (Use as `flag_name="value"` or `flag_name=number`)

- `seed=12345` - **Random seed** for stochastic modes (DARE, etc.)
- `memo="My merge description"` - **Add description** to metadata
- `device="cuda"` - **Use GPU** instead of CPU (much faster!)
- `fine="key_name"` - **Finetune specific keys** on model 0
- `m0_name="Custom Model 0"` - **Custom name** for model 0 in metadata
- `m1_name="Custom Model 1"` - **Custom name** for model 1 in metadata
- `m2_name="Custom Model 2"` - **Custom name** for model 2 in metadata

#### **Random Parameter Flags** (Advanced)

- `rand_alpha="0.2,0.8,42"` - **Randomize alpha** (min, max, seed)
- `rand_beta="0.1,0.5,123"` - **Randomize beta** (min, max, seed)

### 💡 **Flag Usage Examples**

```python
# Basic merge with force overwrite
run_merge("WS", model_1, model_2, alpha=0.4, force=True)

# GPU-accelerated merge with custom metadata
run_merge("SIG", model_1, model_2, alpha=0.6, 
          device="cuda", memo="High quality sigmoid blend")

# Advanced 3-model merge favoring first model's structure
run_merge("sAD", model_1, model_2, model_3, alpha=0.5, beta=0.3,
          cosine0=True, seed=42)

# Experimental merge with cleanup
run_merge("DARE", model_1, model_2, alpha=0.4, beta=0.3,
          delete_source=True, no_metadata=True)
```

### 🎯 **Pro Tips**

1. Start with **WS** or **SIG** for your first merges - they're predictable and stable
2. Use **GEO** when merging very different styles (like anime + realism)
3. **AD** and **sAD** are great for adding specific features from a third model
4. Always backup your models before experimenting!
5. Try different alpha/beta values - small changes can make big differences
6. Use `device="cuda"` if you have a GPU - it's **much faster**!
7. The `cosine` flags are powerful for preserving one model's structure

## 💅 **VAE Merging** — Enhance Your Generation Quality {#vae-merging}

VAE (Variational Autoencoder) merging allows you to combine different VAEs to achieve unique visual characteristics and improved generation quality. Each VAE has its own color profile, contrast handling, and detail enhancement properties.

### **Available VAE Merge Modes:**

| Mode | Description | Use Case |

In [ ]:
# 💜 VAE Merging Setup
print("🎨 Setting up VAE merging...")

# List available VAEs
print(f"📁 Available VAEs ({len(available_vaes)}):")
for i, vae in enumerate(available_vaes):
    print(f"  {i+1:2d}. {vae}")

# Pre-select some VAEs for demo (users can modify these)
if len(available_vaes) >= 2:
    selected_vae_1_idx = 0
    selected_vae_2_idx = 1
    selected_vae_1 = available_vaes[selected_vae_1_idx]
    selected_vae_2 = available_vaes[selected_vae_2_idx]
    
    print(f"\n✨ Default VAE selections for easy testing:")
    print(f"  VAE 1: {selected_vae_1}")
    print(f"  VAE 2: {selected_vae_2}")
    print(f"\n💡 Tip: Modify the indices below to select different VAEs")
else:
    selected_vae_1 = available_vaes[0] if available_vaes else "No VAEs found"
    selected_vae_2 = "Need at least 2 VAEs for merging"
    print(f"\n⚠️  Need at least 2 VAEs for merging. Found: {len(available_vaes)}")

print("✅ VAE merging setup complete!")

### 🌟 **Quick VAE Merge - Weighted Sum**
Perfect for simple VAE blending with a single alpha value.

In [ ]:
# Quick VAE Weighted Sum merge
print("🌟 Quick VAE Merge - Weighted Sum")

# VAE selection (modify these indices to select different VAEs)
vae_1_idx = 0  # Change this to select a different first VAE
vae_2_idx = 1  # Change this to select a different second VAE

# Merge parameters (modify these as needed)
alpha = 0.5  # 0.0 = full VAE 1, 1.0 = full VAE 2
output_name = "merged_vae_ws"

# Validate selections
if len(available_vaes) < 2:
    print(f"❌ Need at least 2 VAEs for merging. Found: {len(available_vaes)}")
elif vae_1_idx >= len(available_vaes) or vae_2_idx >= len(available_vaes):
    print(f"❌ Invalid VAE indices. Available: 0-{len(available_vaes)-1}")
else:
    vae_1_name = available_vaes[vae_1_idx]
    vae_2_name = available_vaes[vae_2_idx]
    
    print(f"\n🎨 VAE Merge Configuration:")
    print(f"  Mode: Weighted Sum (WS)")
    print(f"  VAE 1: {vae_1_name} (index {vae_1_idx})")
    print(f"  VAE 2: {vae_2_name} (index {vae_2_idx})")
    print(f"  Alpha: {alpha} (0.0 = full VAE 1, 1.0 = full VAE 2)")
    print(f"  Output: {output_name}")
    
    # Build and execute command
    vae_cmd = [
        str(venv_python), "lib/merge_vae.py", "WS",
        vae_1_name, vae_2_name,
        "--alpha", str(alpha),
        "--vae-path", vae_path,
        "--output", output_name,
        "--save-safetensors"
    ]
    
    print(f"\n🚀 Starting VAE merge...")
    print(f"Command: {' '.join(vae_cmd)}")
    
    # Execute merge
    result = run_process_safely(vae_cmd, "VAE merge")

### 🎭 **Advanced VAE Merge - Component Specific**
For precise control over encoder and decoder blending ratios.

In [ ]:
# Advanced VAE Component-Specific merge
print("🎭 Advanced VAE Merge - Component Specific")
print("💡 This mode allows different blend ratios for encoder vs decoder components")

# Select VAEs for merging
vae_1_name, vae_1_idx = select_vae("🎨 Select first VAE:", 0)
if vae_1_name is None:
    print("❌ VAE selection cancelled")
else:
    vae_2_name, vae_2_idx = select_vae("🎨 Select second VAE:", 1 if len(available_vaes) > 1 else 0)
    
    if vae_2_name is None:
        print("❌ VAE selection cancelled")
    else:
        try:
            # Get encoder alpha
            encoder_input = input("🔧 Encoder blend ratio (0.0-1.0, default 0.3): ").strip()
            encoder_alpha = float(encoder_input) if encoder_input else 0.3
            
            # Get decoder alpha  
            decoder_input = input("🔧 Decoder blend ratio (0.0-1.0, default 0.7): ").strip()
            decoder_alpha = float(decoder_input) if decoder_input else 0.7
            
            # Validate alphas
            if not (0.0 <= encoder_alpha <= 1.0):
                print("❌ Encoder alpha must be between 0.0 and 1.0, using 0.3")
                encoder_alpha = 0.3
                
            if not (0.0 <= decoder_alpha <= 1.0):
                print("❌ Decoder alpha must be between 0.0 and 1.0, using 0.7") 
                decoder_alpha = 0.7
            
            # Output name
            output_name = input("📝 Output name (default: merged_vae_comp): ").strip() or "merged_vae_comp"
            
            print(f"\n🎭 VAE Component Merge Configuration:")
            print(f"  Mode: Component Specific (COMP)")
            print(f"  VAE 1: {vae_1_name}")
            print(f"  VAE 2: {vae_2_name}")
            print(f"  Encoder α: {encoder_alpha} (how much VAE 2 encoder to blend in)")
            print(f"  Decoder α: {decoder_alpha} (how much VAE 2 decoder to blend in)")
            print(f"  Output: {output_name}")
            
            # Explain the settings
            print(f"\n💡 This configuration will:")
            print(f"  • Use {(1-encoder_alpha)*100:.0f}% VAE 1 + {encoder_alpha*100:.0f}% VAE 2 for encoding")
            print(f"  • Use {(1-decoder_alpha)*100:.0f}% VAE 1 + {decoder_alpha*100:.0f}% VAE 2 for decoding")
            
            # Confirm merge
            confirm = input("\n✨ Proceed with component-specific VAE merge? (Y/n): ").strip().lower()
            if confirm in ['', 'y', 'yes']:
                
                # Build command
                vae_cmd = [
                    str(venv_python), "lib/merge_vae.py", "COMP",
                    vae_1_name, vae_2_name,
                    "--encoder-alpha", str(encoder_alpha),
                    "--decoder-alpha", str(decoder_alpha),
                    "--vae-path", vae_path,
                    "--output", output_name,
                    "--save-safetensors"
                ]
                
                print(f"\n🚀 Starting component-specific VAE merge...")
                print(f"Running: {' '.join(vae_cmd)}")
                
                # Execute merge
                result = run_process_safely(vae_cmd, "Component VAE merge")
                
            else:
                print("🚫 VAE merge cancelled")
                
        except ValueError:
            print("❌ Invalid alpha value, please enter numbers between 0.0 and 1.0")
        except KeyboardInterrupt:
            print("\n🚫 VAE merge cancelled")

### 🔮 **Experimental VAE Merge - SLERP**
Spherical Linear Interpolation for the smoothest possible VAE blending.

In [ ]:
# Experimental SLERP VAE merge
print("🔮 Experimental VAE Merge - SLERP")
print("💡 Spherical Linear Interpolation provides the smoothest blending in high-dimensional space")
print("⚠️  This is computationally intensive but produces the most natural transitions")

# Select VAEs for merging
vae_1_name, vae_1_idx = select_vae("🎨 Select first VAE:", 0)
if vae_1_name is None:
    print("❌ VAE selection cancelled")
else:
    vae_2_name, vae_2_idx = select_vae("🎨 Select second VAE:", 1 if len(available_vaes) > 1 else 0)
    
    if vae_2_name is None:
        print("❌ VAE selection cancelled")
    else:
        try:
            # Get merge ratio
            alpha_input = input("⚖️ SLERP interpolation ratio (0.0-1.0, default 0.5): ").strip()
            alpha = float(alpha_input) if alpha_input else 0.5
            
            if not (0.0 <= alpha <= 1.0):
                print("❌ Alpha must be between 0.0 and 1.0, using 0.5")
                alpha = 0.5
            
            # Output name
            output_name = input("📝 Output name (default: merged_vae_slerp): ").strip() or "merged_vae_slerp"
            
            # Half precision option for SLERP
            half_precision = input("💾 Use half precision to save memory? (Y/n): ").strip().lower()
            use_half = half_precision in ['', 'y', 'yes']
            
            print(f"\n🔮 SLERP VAE Merge Configuration:")
            print(f"  Mode: Spherical Linear Interpolation (SLERP)")
            print(f"  VAE 1: {vae_1_name}")
            print(f"  VAE 2: {vae_2_name}")
            print(f"  Alpha: {alpha} (interpolation point along the sphere)")
            print(f"  Half Precision: {use_half}")
            print(f"  Output: {output_name}")
            
            print(f"\n💡 SLERP blending characteristics:")
            print(f"  • More natural transitions than linear blending")
            print(f"  • Preserves the 'magnitude' of features better")
            print(f"  • Computationally intensive but higher quality")
            print(f"  • Falls back to linear blending if SLERP fails on any tensor")
            
            # Confirm merge
            confirm = input("\n✨ Proceed with SLERP VAE merge? (Y/n): ").strip().lower()
            if confirm in ['', 'y', 'yes']:
                
                # Build command
                vae_cmd = [
                    str(venv_python), "lib/merge_vae.py", "SLERP",
                    vae_1_name, vae_2_name,
                    "--alpha", str(alpha),
                    "--vae-path", vae_path,
                    "--output", output_name,
                    "--save-safetensors"
                ]
                
                if use_half:
                    vae_cmd.append("--save-half")
                
                print(f"\n🚀 Starting SLERP VAE merge...")
                print(f"⏳ Note: This may take longer than other merge methods")
                print(f"Running: {' '.join(vae_cmd)}")
                
                # Execute merge
                result = run_process_safely(vae_cmd, "SLERP VAE merge")
                
            else:
                print("🚫 VAE merge cancelled")
                
        except ValueError:
            print("❌ Invalid alpha value, please enter a number between 0.0 and 1.0")
        except KeyboardInterrupt:
            print("\n🚫 VAE merge cancelled")